In [1]:
from CTRModules import *

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader

from torchmetrics.classification import AUROC

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import gc

import matplotlib.pyplot as plt

In [2]:
DATA_PATH = "F:/CTR_Prediction/"

X_train = pd.read_csv(DATA_PATH + "X_train.csv")
X_val = pd.read_csv(DATA_PATH + "X_val.csv")
y_train = pd.read_csv(DATA_PATH + "y_train.csv")
y_val = pd.read_csv(DATA_PATH + "y_val.csv")

In [3]:
X_train_seq =  X_train.pop("seq")
X_val_seq = X_val.pop("seq")

In [4]:
BATCH_SIZE = 128

trainDS = CTRDataset(X_train, X_train_seq, y_train)
valDS = CTRDataset(X_val, X_val_seq, y_val)

trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn_train)
valDL = DataLoader(valDS, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_train)

In [5]:
EPOCH = 300
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 1e-4

hidden_dim = [512, 256, 128]

num_embeddings = 600 + 1
transformer_dim = 32
nhead = 4
num_encoder_layers = 1

model = DCNv2(input_dim=X_train.shape[1],
              num_layers=2,
              hidden_dim=hidden_dim,
              dropout=0.2,
              num_embeddings=num_embeddings,
              transformer_dim=transformer_dim,
              nhead=nhead,
              num_encoder_layers=num_encoder_layers
              ).to(DEVICE)

optimizer = optim.AdamW(model.parameters(), lr=LR)

weight = torch.tensor([2])

loss_fn = nn.BCELoss(weight=weight).to(DEVICE)
score_fn = AUROC(task="binary")

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5)

data_size = len(X_train)
val_data_size = len(X_val)

In [6]:
del X_train, X_train_seq, X_val, X_val_seq, y_train, y_val

gc.collect()
torch.cuda.empty_cache()

In [7]:
loss, score = training(model, trainDL, valDL, optimizer, EPOCH,
                       data_size, val_data_size, loss_fn, score_fn, 
                       scheduler, DEVICE)

[1 / 300]
 - TRAIN LOSS : 89.73727512125107
- TRAIN DICE SCORE : 0.4804210364818573

 - TEST LOSS : 111.89199812461227
- TEST DICE SCORE : 0.46169114112854004
[2 / 300]
 - TRAIN LOSS : 125.97684727436841
- TRAIN DICE SCORE : 0.47988927364349365

 - TEST LOSS : 133.2402781864126
- TEST DICE SCORE : 0.5000737309455872
[3 / 300]
 - TRAIN LOSS : 133.25769746430473
- TRAIN DICE SCORE : 0.4999973177909851

 - TEST LOSS : 133.23701309352072
- TEST DICE SCORE : 0.5001283884048462
[4 / 300]
 - TRAIN LOSS : 133.25771802584754
- TRAIN DICE SCORE : 0.5000008940696716

 - TEST LOSS : 133.23701309352072
- TEST DICE SCORE : 0.5001283884048462


KeyboardInterrupt: 

NameError: name 'X_train' is not defined

In [8]:
TEST_PATH = "./data/test.parquet"

test = pd.read_parquet(TEST_PATH, engine="pyarrow").drop(columns=['ID'])

In [10]:
test_seq =  test.pop("seq")

In [22]:
test = test.astype(np.float32)

In [27]:
test.shape

(1527298, 117)

In [23]:
testDS = CTRDataset(test, test_seq)

testDL = DataLoader(testDS, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_test)

In [24]:
model.eval()
output = []

with torch.no_grad():
    for test_features, test_seq, _ in testDL:
        test_features, test_seq = test_features.to(DEVICE), test_seq.to(DEVICE)
        output.append(model(test_features, test_seq).cpu())

test_preds = torch.cat(output).numpy()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x149 and 245x245)

In [ ]:
print(test_preds[:10])

In [ ]:
SUBMISSION_PATH = "./data/sample_submission.csv"

submission_df = pd.read_csv(SUBMISSION_PATH)

submission_df["clicked"] = test_preds


In [ ]:
submission_df.to_csv("./data/DCNv2_submission.csv", index=False)